In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

#read train_texts ...

#English readability

In [ ]:
!pip install readability
import readability

In [ ]:
import csv

for i, text in enumerate(train_texts):
  results = readability.getmeasures(text, lang='en')
  with open(r'', 'a') as f:
    writer = csv.writer(f)
    writer.writerow([results['readability grades']['Kincaid'],\
                 results['readability grades']['Coleman-Liau'],\
                 results['readability grades']['ARI'],\
                 results['readability grades']['SMOGIndex'],\
                 results['readability grades']['DaleChallIndex']])

#Morphological

In [ ]:
#preprocessing
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer 

import re
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

!pip install pymorphy2
import pymorphy2, re
ma = pymorphy2.MorphAnalyzer()

def clean_text(text):#rus
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ").replace("«", '').replace("»", '').replace("…", '')
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) 
    text = re.sub('[.,:;<>_%©?–*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text)
    text = " ".join(ma.parse(word)[0].normal_form for word in text.split())
    new_text = []
    for word in text.split():
      if word not in stops:
        new_text.append(word)
    return ' '.join(new_text)

def clean_text_eng(text):#eng
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ").replace("«", ' ').replace("»", ' ').replace("…", ' ')
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', ' ', text) 
    text = re.sub('[.,:;<>_%©?–*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    #text = " ".join(lemmatizer.lemmatize(word) for word in text.split())
    #text = " ".join(ma.parse(word)[0].normal_form for word in text.split())
    '''
    new_text = []
    for word in text.split():
      if word not in stops:
        new_text.append(word)
    
    return ' '.join(new_text)
    '''
    return text

In [ ]:
import numpy as np
import csv

pos_list = ['CC', 'CD', 'DT', 'FW', 'EX', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',\
            'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',\
            'WDT', 'WP', 'WRB', 'WP$']
for j, text in enumerate(train_texts):
  counts = np.zeros(len(pos_list))
  text = clean_text_eng(text)
  tokens=nltk.word_tokenize(text)
  tokens = nltk.pos_tag(tokens)
  for i, token in enumerate(tokens):
    try:
      pos = pos_list.index(tokens[i][1])
      counts[pos] += 1
    except:
      print(tokens[i][1])
  counts = [el/np.sum(counts) for el in counts]
  with open(r'', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(counts)
    
for j, text in enumerate(test_texts):
  counts = np.zeros(len(pos_list))
  text = clean_text_eng(text)
  tokens=nltk.word_tokenize(text)
  tokens = nltk.pos_tag(tokens)
  for i, token in enumerate(tokens):
    try:
      pos = pos_list.index(tokens[i][1])
      counts[pos] += 1
    except:
      print(tokens[i][1])
  counts = [el/np.sum(counts) for el in counts]
  with open(r'', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(counts)

In [ ]:
import numpy as np
pos = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB',\
       'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']
path = ''

import csv   

#'''
labels = ['filename','morpho_NOUN', 'morpho_ADJF', 'morpho_ADJS', 'morpho_COMP', 'morpho_VERB', 'morpho_INFN', 'morpho_PRTF', 'morpho_PRTS', 'morpho_GRND',\
          'morpho_NUMR', 'morpho_ADVB', 'morpho_NPRO', 'morpho_PRED', 'morpho_PREP', 'morpho_CONJ', 'morpho_PRCL', 'morpho_INTJ',\
          'morpho_FUTR', 'morpho_PRES', 'morpho_PAST', 'morpho_ANIMACY', 'morph_PERF', 'morpho_TRANS', 'morpho_NOMN', 'morpho_GENT', 'morpho_DATV',\
          'morpho_ACCS', 'morpho_ABLT', 'morpho_LOCT', 'morpho_VOCT', 'morpho_GEN2', 'morpho_ACC2', 'morpho_LOC2']

with open(r'', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(labels)
#'''

found = True

'''
for i, f in enumerate(files):
  if found:
    with open(path + f) as f1:
        content = f1.read()
      '''
for i, text in enumerate(train_texts):
    pos_count = np.zeros(len(pos) + 6)
    #words = clean_text(content).split()
    words = clean_text(text).split()

    odush = 0
    perf = 0
    trans = 0
    num_verbs_personal = 0
    past = 0
    pres = 0
    futr = 0

    cases = np.zeros(10)

    for word in words:
      try:
        p = ma.parse(word)[0]
        pos_tag = p.tag.POS
        pos_count[pos.index(pos_tag)]+=1
        if pos_tag == 'NOUN':
          if p.tag.animacy == 'anim':
            odush += 1

          if p.tag.case == 'nomn':
            cases[0] += 1
          elif p.tag.case == 'gent':
            cases[1] += 1
          elif p.tag.case == 'datv':
            cases[2] += 1
          elif p.tag.case == 'accs':
            cases[3] += 1
          elif p.tag.case == 'ablt':
            cases[4] += 1
          elif p.tag.case == 'loct':
            cases[5] += 1
          elif p.tag.case == 'voct':
            cases[6] += 1
          elif p.tag.case == 'gen2':
            cases[7] += 1
          elif p.tag.case == 'acc2':
            cases[8] += 1
          elif p.tag.case == 'loc2':
            cases[9] += 1

        if pos_tag == 'VERB' or pos_tag == 'INFN':
          if p.tag.aspect == 'perf':
            perf += 1
          if p.tag.transitivity == 'tran':
            trans += 1
          if p.tag.tense == 'futr':
            futr += 1
            num_verbs_personal += 1
          elif p.tag.tense == 'past':
            past += 1
            num_verbs_personal += 1
          elif p.tag.tense == 'pres':
            pres += 1
            num_verbs_personal += 1
        #считаем одушевленные
      except:
        pass

    num_nouns = pos_count[0]
    num_verbs = pos_count[4] + pos_count[5]

    pos_count = pos_count / len(words)
    pos_count[len(pos_count)-1] = trans / num_verbs
    pos_count[len(pos_count)-2] = perf / num_verbs
    pos_count[len(pos_count)-3] = odush / num_nouns
    if num_verbs_personal!=0:
      pos_count[len(pos_count)-4] = past / num_verbs_personal
      pos_count[len(pos_count)-5] = pres / num_verbs_personal
      pos_count[len(pos_count)-6] = futr / num_verbs_personal
    else:
      pos_count[len(pos_count)-4] = 0
      pos_count[len(pos_count)-5] = 0
      pos_count[len(pos_count)-6] = 0

    res = [f]
    res.extend(list(np.concatenate((pos_count,cases/num_nouns))))

    #print(np.concatenate((pos_count,cases/num_nouns)))
    with open(r'', 'a') as f2:
      writer = csv.writer(f2)
      writer.writerow(res)

    if i%1000==0:
      print(i)

#Russian readability

In [ ]:
import requests

import csv  

found = True

path = ""
labels = ["filename", "readability_index_cl", "readability_index_ari", "readability_index_dc", "readability_index_SMOG", "readability_index_fk"]

with open(r'', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(labels)

for i, text in enumerate(train_texts):
    try:
      content = text
      response = requests.post("https://api.plainrussian.ru/api/1.0/ru/measure/", data={"text":content}).json()
      new_line = [f, response['indexes']['index_cl'], response['indexes']['index_ari'], response['indexes']['index_dc'], \
                  response['indexes']['index_SMOG'], response['indexes']['index_fk']]

      with open(r'', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(new_line)

      if i%1000==0:
          print(i)
    except:
      print(i, f)
      new_line = [f]
      with open(r'', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(new_line)

#LDA

In [ ]:
import re
import numpy as np
import pandas as pd

import nltk
nltk.download('stopwords')
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
train_texts = [clean_text_eng(text) for text in train_texts]
test_texts = [clean_text_eng(text) for text in test_texts]
train_texts = [el.split() for el in train_texts]
test_texts = [el.split() for el in test_texts]

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(train_texts, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[train_texts], threshold=100)  
bigram_ = gensim.models.Phrases(test_texts, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_ = gensim.models.Phrases(bigram[test_texts], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
bigram_mod_ = gensim.models.phrases.Phraser(bigram_)
trigram_mod_ = gensim.models.phrases.Phraser(trigram_)

# See trigram example
print(trigram_mod[bigram_mod[train_texts[0]]])

In [ ]:
import spacy
# Define functions for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(train_texts)
data_words_nostops_ = remove_stopwords(test_texts)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_bigrams_ = make_bigrams(data_words_nostops_)

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
corpus_ = [id2word.doc2bow(text) for text in data_words_bigrams_]

# View
print(corpus[:1])

In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=100, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           workers=3,
                                           per_word_topics=True)

In [ ]:
def get_dist(dist):
  new_dist = []
  for d in dist:
    new_dist.append(d[1])
  return new_dist

for doc in corpus:
  with open(r'', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(get_dist(lda_model.get_document_topics(doc, minimum_probability=0.0)))

for doc in corpus_:
  with open(r'', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(get_dist(lda_model.get_document_topics(doc, minimum_probability=0.0)))